In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
async def get_expiry(f_today, index):

    if index == 'bnf':    
        if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
            f_expiry = dt.date(2024, 1, 25)
        elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
            f_expiry = dt.date(2024, 1, 31)
        elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
            f_expiry = dt.date(2024, 2, 29)
        elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
            f_expiry = dt.date(2024, 2, 27)
        elif f_today < dt.date(2023, 9, 1):
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
        elif f_today >= dt.date(2023, 9, 1):
            if f_today.day < 24:
                days_to_wednesday = (2 - f_today.weekday()) % 7
                nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
                f_expiry = nearest_wednesday
                if nse.valid_days(
                    start_date=nearest_wednesday, end_date=nearest_wednesday
                ).empty:
                    f_expiry = nearest_wednesday - dt.timedelta(days=1)
            else:
                days_to_thursday = (3 - f_today.weekday()) % 7
                nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
                f_expiry = nearest_thursday
                if nse.valid_days(
                    start_date=nearest_thursday, end_date=nearest_thursday
                ).empty:
                    f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'nifty':
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'finnifty' or index == 'fnf':
        days_to_thursday = (1 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'midcpnifty' or index == 'midcp':
        days_to_thursday = (0 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [5]:
bnf_1min = pd.read_csv("../data/bnf_wave.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["time"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2017]

In [6]:
bnf_1min.tail()

,time,open,high,low,close,MA,Plot,Zero line,MACD Signal,datetime
22214,2024-12-05T11:45:00+05:30,53037.0000,53507.1000,52998.8500,53484.5000,53167.1400,204.1806,0,204.1806,2024-12-05 11:45:00+05:30
22215,2024-12-05T12:15:00+05:30,53475.8000,53647.8000,53472.6500,53521.0500,53191.2367,270.8858,0,270.8858,2024-12-05 12:15:00+05:30
22216,2024-12-05T12:45:00+05:30,53523.2500,53621.1000,53452.6500,53609.9500,53230.6600,327.7843,0,327.7843,2024-12-05 12:45:00+05:30
22217,2024-12-05T13:15:00+05:30,53614.8000,53642.3500,53461.3500,53598.9000,53269.0200,351.5606,0,351.5606,2024-12-05 13:15:00+05:30
22218,2024-12-05T13:45:00+05:30,53599.7500,53640.9000,53560.0000,53564.1500,53299.1367,348.2676,0,348.2676,2024-12-05 13:45:00+05:30


In [7]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [8]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2024, 7, 30),
 datetime.date(2019, 7, 25),
 datetime.date(2020, 4, 8),
 datetime.date(2021, 2, 12),
 datetime.date(2019, 8, 30),
 datetime.date(2019, 2, 15),
 datetime.date(2021, 4, 19),
 datetime.date(2020, 11, 23),
 datetime.date(2020, 2, 18),
 datetime.date(2020, 7, 27),
 datetime.date(2023, 8, 9),
 datetime.date(2022, 10, 19),
 datetime.date(2020, 10, 15),
 datetime.date(2024, 1, 24),
 datetime.date(2022, 8, 19),
 datetime.date(2024, 6, 24),
 datetime.date(2023, 11, 22),
 datetime.date(2019, 1, 1),
 datetime.date(2021, 12, 31),
 datetime.date(2021, 12, 30),
 datetime.date(2020, 7, 23),
 datetime.date(2019, 1, 23),
 datetime.date(2018, 12, 21),
 datetime.date(2018, 9, 7),
 datetime.date(2021, 3, 8),
 datetime.date(2020, 9, 8),
 datetime.date(2018, 11, 5),
 datetime.date(2019, 3, 29),
 datetime.date(2024, 11, 1),
 datetime.date(2021, 2, 11),
 datetime.date(2020, 4, 16),
 datetime.date(2022, 8, 26),
 datetime.date(2023, 9, 14),
 datetime.date(2018, 11, 27),
 datetime.da

In [10]:
PORTFOLIO_VALUE = 10_00_000 # 10 Lacs
# RPT_PCT = 0.01 # 1% RPT
SLIPPAGE_ = 0.0001
LEVERAGE_ = 5

In [529]:
import pandas as pd

def calculate_signals(df, n, rsi_period, rsi_overbought, ema_length):
    # Calculate RSI
    def calculate_rsi(series, period):
        # Calculate the price changes
        delta = series.diff()
    
        # Separate gains and losses
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
    
        # Calculate the first average gain and loss (Simple Moving Average for the first period)
        avg_gain = gain.rolling(window=period, min_periods=period).mean()
        avg_loss = loss.rolling(window=period, min_periods=period).mean()
    
        # Calculate the subsequent smoothed averages (similar to EMA)
        for i in range(period, len(series)):
            avg_gain.iloc[i] = (avg_gain.iloc[i - 1] * (period - 1) + gain.iloc[i]) / period
            avg_loss.iloc[i] = (avg_loss.iloc[i - 1] * (period - 1) + loss.iloc[i]) / period
    
        # Calculate the Relative Strength (RS) and the RSI
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
    
        return rsi

    # Apply the TradingView-style RSI calculation
    df['RSI'] = calculate_rsi(df['close'], rsi_period)

    # Calculate the previous n candles' low
    df['Prev_N_Low'] = df['low'].rolling(window=n).min().shift(1)
    df['ema'] = df['close'].ewm(span=ema_length, adjust=False).mean()

    # Signal generation logic
    df['Sell_Signal'] = (
        (df['close'] < df['Prev_N_Low']) &
        (df['RSI'] > rsi_overbought)
    )

    return df

In [530]:
bnf_1min.head()

,time,open,high,low,close,MA,Plot,Zero line,MACD Signal,datetime,RSI,Prev_N_Low,ema,Sell_Signal
0,2018-01-01T09:15:00+05:30,25566.4500,25588.0000,25496.5500,25551.5000,NaN,NaN,0,NaN,2018-01-01 09:15:00+05:30,NaN,NaN,25551.5000,False
1,2018-01-01T09:45:00+05:30,25551.7000,25574.2500,25537.1500,25561.5000,NaN,NaN,0,NaN,2018-01-01 09:45:00+05:30,NaN,NaN,25553.3182,False
2,2018-01-01T10:15:00+05:30,25563.6500,25573.2500,25541.4000,25554.0500,NaN,NaN,0,NaN,2018-01-01 10:15:00+05:30,NaN,NaN,25553.4512,False
3,2018-01-01T10:45:00+05:30,25556.3000,25562.6500,25545.0500,25552.6500,NaN,NaN,0,NaN,2018-01-01 10:45:00+05:30,NaN,25496.5500,25553.3056,False
4,2018-01-01T11:15:00+05:30,25555.0000,25566.3000,25547.8500,25551.8000,NaN,NaN,0,NaN,2018-01-01 11:15:00+05:30,NaN,25537.1500,25553.0318,False


In [531]:
lev_ = 5
portfolio_value = 10000000
lot_size_ = 15
slippage_ = 0.0001

In [532]:
def backtest_short(df, n, rsi_period, rsi_overbought, ema_length):
    trade_book = []
    in_trade = False
    points = 0
    is_trailing_active = False
    trailing_sl = 0
    df = calculate_signals(df, n, rsi_period, rsi_overbought, ema_length)
    # print(df[df['Sell_Signal']].to_string())
    for i in range(n, len(df)):
        # print(i, df.loc[i], df.loc[i, 'open'])
        # break
        if df.loc[i, 'Sell_Signal'] and not in_trade:
            # Entry Triggered
            entry_price = df.loc[i, 'close']
            entry_time = df.loc[i, 'datetime']
            initial_sl = df.loc[i-n:i, 'high'].max()
            sl_in_points = initial_sl - entry_price
            target = 5 * sl_in_points
            in_trade = True

        if in_trade:
            if df.loc[i, 'high'] < df.loc[i, 'ema']:
                is_trailing_active = True
            if is_trailing_active:
                trailing_sl = df.loc[i, 'ema']
            # print(df.loc[i, 'open'], initial_sl)
            # break
            if df.loc[i, 'open'] > initial_sl and not points:
                # Gap condition
                if df.loc[i, 'close'] < initial_sl:
                    initial_sl = df.loc[i, 'high']
                else:
                    exit_price = df.loc[i, 'close']
                    exit_time = df.loc[i, 'datetime']
                    in_trade = False
                    remark = 'Gap SL'
                    points = entry_price - exit_price
            if df.loc[i, 'high'] > initial_sl and not points:
                # Initial SL Hit
                exit_price = df.loc[i, 'close']
                exit_time = df.loc[i, 'datetime']
                in_trade = False
                remark = 'Initial SL'
                points = entry_price - exit_price
            if is_trailing_active:
                if df.loc[i, 'close'] > trailing_sl and not points:
                    # TSL Hit
                    exit_price = df.loc[i, 'close']
                    exit_time = df.loc[i, 'datetime']
                    in_trade = False
                    remark = 'Trailing SL'
                    points = entry_price - exit_price
            if df.loc[i, 'low'] < entry_price - target and not points:
                # Target Hit
                exit_price = entry_price - target
                exit_time = df.loc[i, 'datetime']
                in_trade = False
                remark = 'Target'
                points = entry_price - exit_price

        if points:
            index_lev = lev_
            qty = int(round(portfolio_value * index_lev / entry_price / lot_size_)) * lot_size_
            slippage = slippage_ * (entry_price + exit_price)
            final_points = points - slippage
            trade = {
                "Trade Type": "Short",
                "Entry Time": entry_time,
                "Entry Price": entry_price,
                "Initial SL": initial_sl,
                "Trailing SL": trailing_sl,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Points Captured": points,
                "Slippage in Points": slippage,
                "After Costs": final_points,
                "PnL": final_points * qty,
                "Remarks": remark,
                "Qty": qty,
                "Leverage": index_lev,
                "ROI%": (final_points * qty / portfolio_value) * 100,
                "Trade Year": entry_time.year,
                "Trade Month": entry_time.month,
            }
            # print(trade)
            trade_book.append(trade)
            points = 0
            in_trade = False
            is_trailing_active = False
            trailing_sl = 0

    return pd.DataFrame(trade_book)
            

In [543]:
tb = backtest_short(bnf_1min, n=3, rsi_period=14, rsi_overbought=40, ema_length=20)

In [544]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2018, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2018, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [545]:
variation = 1
stats = generate_stats(tb, variation)
for x, y in stats.items():
    final_stats = y

final_stats

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2018,29.0533,111,40.5405,3.9648,-2.2631,-42.0875,0.6903,1
2019,79.3232,118,40.6780,4.8162,-2.1694,-39.9548,1.9853,1
2020,88.8128,110,43.6364,9.7239,-6.0957,-67.4933,1.3159,1
2021,-3.9658,124,34.6774,5.7466,-3.0996,-71.4137,-0.0555,1
2022,-23.9943,135,34.8148,4.8382,-2.8567,-118.1220,-0.2031,1
2023,-49.7776,119,32.7731,3.3859,-2.2729,-81.2556,-0.6126,1
2024,3.7258,100,36.0000,3.9044,-2.1380,-37.9056,0.0983,1
Overall,123.1774,817.0000,37.4541,5.3054,-2.9360,-169.4186,0.7271,1.0000


In [546]:
tb[tb['Trade Year'] > 2023]

,Trade Type,Entry Time,Entry Price,Initial SL,Trailing SL,Exit Time,Exit Price,Points Captured,Slippage in Points,After Costs,PnL,Remarks,Qty,Leverage,ROI%,Trade Year,Trade Month
717,Short,2024-01-01 14:45:00+05:30,48213.6500,48450.0000,47779.7750,2024-01-04 09:15:00+05:30,47892.8500,320.8000,9.6106,311.1894,322080.9773,Trailing SL,1035,5,3.2208,2024,1
718,Short,2024-01-05 10:15:00+05:30,48166.6500,48381.9500,48067.8811,2024-01-05 14:45:00+05:30,48167.6000,-0.9500,9.6334,-10.5834,-10953.8449,Trailing SL,1035,5,-0.1095,2024,1
719,Short,2024-01-08 10:45:00+05:30,47874.5000,48154.5000,47785.8256,2024-01-09 10:15:00+05:30,47882.7500,-8.2500,9.5757,-17.8257,-18717.0112,Trailing SL,1050,5,-0.1872,2024,1
720,Short,2024-01-09 13:45:00+05:30,47674.8000,47877.0500,47410.4392,2024-01-11 09:15:00+05:30,47503.5000,171.3000,9.5178,161.7822,169871.2785,Trailing SL,1050,5,1.6987,2024,1
721,Short,2024-01-12 14:45:00+05:30,47659.9500,47873.7000,0.0000,2024-01-15 09:15:00+05:30,48073.5500,-413.6000,9.5733,-423.1734,-444332.0175,Initial SL,1050,5,-4.4433,2024,1
722,Short,2024-01-16 12:45:00+05:30,48122.8500,48305.4000,48005.5157,2024-01-17 09:15:00+05:30,47210.1000,912.7500,9.5333,903.2167,934829.2897,Target,1035,5,9.3483,2024,1
723,Short,2024-01-19 10:45:00+05:30,45930.1500,46249.8500,45892.6035,2024-01-20 10:15:00+05:30,45928.1000,2.0500,9.1858,-7.1358,-7813.7284,Trailing SL,1095,5,-0.0781,2024,1
724,Short,2024-01-20 12:15:00+05:30,45733.0500,46006.0500,0.0000,2024-01-20 12:45:00+05:30,46078.1500,-345.1000,9.1811,-354.2811,-387937.8264,Initial SL,1095,5,-3.8794,2024,1
725,Short,2024-01-30 14:15:00+05:30,45307.8000,45492.5000,0.0000,2024-01-31 09:45:00+05:30,45824.4500,-516.6500,9.1132,-525.7632,-583597.1797,Initial SL,1110,5,-5.8360,2024,1
726,Short,2024-01-31 12:15:00+05:30,45915.6500,46179.7500,0.0000,2024-02-01 11:45:00+05:30,46098.9500,-183.3000,9.2015,-192.5015,-210789.0987,Initial SL,1095,5,-2.1079,2024,1


In [547]:
stats_dictionary = {}
rsi_lengths = [14, 9, 21, 6, 12, 16]
for i in range(2, 5):
    for j in rsi_lengths:
        for k in range(30, 81, 5):
            for l in range(5, 36, 5):
                variation = f'n {i}, rsi {j}, threshold {k}, ema {l}'
                print(variation)
                tb = backtest_short(bnf_1min, i, j, k, l)
                if len(tb) > 0:
                    stats = generate_stats(tb, variation)
                    for x, y in stats.items():
                        if x > 8:
                            final_stats = y
                            print(final_stats.to_string())
                            stats_dictionary[x] = y


n 2, rsi 14, threshold 30, ema 5
n 2, rsi 14, threshold 30, ema 10
n 2, rsi 14, threshold 30, ema 15
n 2, rsi 14, threshold 30, ema 20
n 2, rsi 14, threshold 30, ema 25
n 2, rsi 14, threshold 30, ema 30
n 2, rsi 14, threshold 30, ema 35
n 2, rsi 14, threshold 35, ema 5
n 2, rsi 14, threshold 35, ema 10
n 2, rsi 14, threshold 35, ema 15
n 2, rsi 14, threshold 35, ema 20
n 2, rsi 14, threshold 35, ema 25
n 2, rsi 14, threshold 35, ema 30
n 2, rsi 14, threshold 35, ema 35
n 2, rsi 14, threshold 40, ema 5
n 2, rsi 14, threshold 40, ema 10
n 2, rsi 14, threshold 40, ema 15
n 2, rsi 14, threshold 40, ema 20
n 2, rsi 14, threshold 40, ema 25
n 2, rsi 14, threshold 40, ema 30
n 2, rsi 14, threshold 40, ema 35
n 2, rsi 14, threshold 45, ema 5
n 2, rsi 14, threshold 45, ema 10
n 2, rsi 14, threshold 45, ema 15
n 2, rsi 14, threshold 45, ema 20
n 2, rsi 14, threshold 45, ema 25
n 2, rsi 14, threshold 45, ema 30
n 2, rsi 14, threshold 45, ema 35
n 2, rsi 14, threshold 50, ema 5
n 2, rsi 14, thresh

/tmp/ipykernel_2712/873016624.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


n 2, rsi 14, threshold 70, ema 10


/tmp/ipykernel_2712/873016624.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


n 2, rsi 14, threshold 70, ema 15


...

n 4, rsi 16, threshold 70, ema 30


/tmp/ipykernel_2712/873016624.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


n 4, rsi 16, threshold 70, ema 35


/tmp/ipykernel_2712/873016624.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


n 4, rsi 16, threshold 75, ema 5
n 4, rsi 16, threshold 75, ema 10
n 4, rsi 16, threshold 75, ema 15
n 4, rsi 16, threshold 75, ema 20
n 4, rsi 16, threshold 75, ema 25
n 4, rsi 16, threshold 75, ema 30
n 4, rsi 16, threshold 75, ema 35
n 4, rsi 16, threshold 80, ema 5
n 4, rsi 16, threshold 80, ema 10
n 4, rsi 16, threshold 80, ema 15
n 4, rsi 16, threshold 80, ema 20
n 4, rsi 16, threshold 80, ema 25
n 4, rsi 16, threshold 80, ema 30
n 4, rsi 16, threshold 80, ema 35


In [548]:
stats_dictionary

{9.625460232986592:         Total ROI Total Trades Win Rate Avg Profit% per Trade  \
 2018       2.6945            2 100.0000                1.3473   
 2019      -0.8128            1   0.0000                   NaN   
 2020       0.0000            0      NaN                   NaN   
 2021       0.0000            0      NaN                   NaN   
 2022       6.3437            1 100.0000                6.3437   
 2023       0.0000            0      NaN                   NaN   
 2024      -0.4016            1   0.0000                   NaN   
 Overall    7.8238            5  60.0000                3.0127   
 
         Avg Loss% per Trade Max Drawdown ROI/DD Ratio  \
 2018                    NaN       0.0000          inf   
 2019                -0.8128       0.0000         -inf   
 2020                    NaN          NaN          NaN   
 2021                    NaN          NaN          NaN   
 2022                    NaN       0.0000          inf   
 2023                    NaN         